In [3]:
# ======================
# 📰 FAKE NEWS DETECTOR
# ======================

# 1️⃣ Install required packages
!pip install streamlit pyngrok transformers torch pandas --quiet

# 2️⃣ Import libraries and configure ngrok
import os
from pyngrok import ngrok, conf

conf.get_default().auth_token = "33eEQATYeFwXNMBlPYBjYqGVq9y_5FTbXiK9V4YT8bUATN2RB"

# Kill previous tunnels if any
ngrok.kill()

# 3️⃣ Write Streamlit app
app_code = r'''
import streamlit as st
from transformers import pipeline
import torch
import pandas as pd

st.set_page_config(page_title="Fake News Detector", page_icon="📰", layout="centered")
st.title("📰 Fake News Detector for Students")
st.write("Analyze any news article or social media post to detect reliability and get a short summary.")

# =====================
# Sidebar
# =====================
st.sidebar.header("⚙️ Options")
max_len = st.sidebar.slider("Max summary length", 40, 200, 80)
show_summary = st.sidebar.checkbox("Show summary", True)
st.sidebar.markdown("---")
st.sidebar.caption("Developed by **Roshan J** | © 2025")

# =====================
# Load Models
# =====================
@st.cache_resource
def load_models():
    device = 0 if torch.cuda.is_available() else -1

    classifier = pipeline(
        "text-classification",
        model="jy46604790/Fake-News-Bert-Detect",
        tokenizer="jy46604790/Fake-News-Bert-Detect",
        device=device
    )

    # Works with your transformers version
    summarizer = pipeline(
        "text-generation",
        model="facebook/bart-large-cnn",
        device=device
    )

    return classifier, summarizer


classifier, summarizer = load_models()

# =====================
# History storage
# =====================
if "history" not in st.session_state:
    st.session_state.history = []

# =====================
# Input
# =====================
text_input = st.text_area("🗞 Paste your news article or post here:", height=250)

# =====================
# Analyze
# =====================
if st.button("🔍 Analyze"):

    if not text_input.strip():
        st.warning("⚠️ Please enter some text first.")

    else:
        with st.spinner("Analyzing..."):

            # Limit text (important)
            text_input = text_input[:1024]

            # Prediction
            result = classifier(text_input)[0]
            label = result["label"]
            score = round(result["score"] * 100, 2)

            verdict = "FAKE" if label == "LABEL_0" else "REAL"

            # Summary
            summary = summarizer(
                text_input,
                max_length=max_len,
                do_sample=False
            )[0]["generated_text"]

        # =====================
        # Output
        # =====================
        st.subheader("📊 Credibility Analysis")

        col1, col2 = st.columns(2)
        col1.metric("Verdict", verdict)
        col2.metric("Confidence", f"{score}%")

        if verdict == "FAKE":
            st.error(f"🚫 FAKE News — Confidence: {score}%")
        else:
            st.success(f"✅ REAL News — Confidence: {score}%")

        if show_summary:
            st.subheader("📝 Summary")
            st.info(summary)

        # Google verify link
        query = "+".join(text_input[:60].split())
        st.markdown(f"[🔎 Verify on Google News](https://news.google.com/search?q={query})")

        # Save history
        st.session_state.history.append({
            "text": text_input[:150] + "..." if len(text_input) > 150 else text_input,
            "label": verdict,
            "score": f"{score}%",
            "summary": summary
        })

# =====================
# History section
# =====================
if st.session_state.history:

    st.markdown("---")
    st.subheader("📜 Analysis History")

    df = pd.DataFrame(st.session_state.history)

    st.download_button(
        "⬇️ Download History CSV",
        df.to_csv(index=False),
        file_name="history.csv",
        mime="text/csv"
    )

    if st.button("🗑️ Clear History"):
        st.session_state.history.clear()
        st.success("History cleared!")

    for i, record in enumerate(reversed(st.session_state.history), 1):
        with st.expander(f"📰 Entry #{i} — {record['label']} ({record['score']})"):
            st.write("**Original:**", record["text"])
            st.write("**Summary:**", record["summary"])

st.markdown("---")
st.caption("Made with ❤️ using Streamlit & Transformers | Developed by Roshan J")
'''

# Save app
with open("app.py", "w") as f:
    f.write(app_code)

# 4️⃣ Run Streamlit
os.system("nohup streamlit run app.py --server.port 8501 > /dev/null 2>&1 &")

# 5️⃣ Start ngrok
public_url = ngrok.connect(8501)
print(f"🚀 Your public URL: {public_url}")


🚀 Your public URL: NgrokTunnel: "https://objurgative-sensationistic-georgiann.ngrok-free.dev" -> "http://localhost:8501"
